<a href="https://colab.research.google.com/github/isaachuahy/projects/blob/main/APM466_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.graph_objects as go

In [31]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def calculate_dirty_prices(bonds, settlement_date_str):
    """
    Calculate dirty prices for bonds with dates in M/D/YYYY format.
    Args:
        bonds (list): List of dictionaries with:
            - 'coupon': Annual coupon rate (e.g., 2.5 for 2.5%)
            - 'maturity_date': M/D/YYYY string
            - 'coupon_payment_start_date': M/D/YYYY string
            - 'close_price': Clean price
        settlement_date_str (str): Settlement date (M/D/YYYY)
    Returns:
        List of bonds with added 'dirty_price' field
    """
    # Parse settlement date
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()

    for bond in bonds:
        # transforming from string objects
        coupon_start = datetime.strptime(
            bond['coupon_payment_start_date'], '%m/%d/%Y'
        ).date()
        maturity = datetime.strptime(
            bond['maturity_date'], '%m/%d/%Y'
        ).date()

        # generating all semi-annual coupon dates, naively
        coupon_dates = []
        current_date = coupon_start
        while current_date <= maturity:
            coupon_dates.append(current_date)
            current_date += relativedelta(months=6)  # add 6 months
            if current_date > maturity:
                break

        # Find last coupon date <= settlement date
        last_coupon = None
        for date in reversed(coupon_dates):
            if date <= settlement_date:
                last_coupon = date
                break

        if not last_coupon:  # no coupons accrued yet
            accrued_interest = 0.0
        else:
            # find next coupon date
            next_coupon_idx = coupon_dates.index(last_coupon) + 1
            next_coupon = coupon_dates[next_coupon_idx] if next_coupon_idx < len(coupon_dates) else maturity

            # days since last coupon & total days in coupon period
            accrued_days = (settlement_date - last_coupon).days
            total_days = 182.5 # Canadian bond convention based on document

            # calculate accrued interest for dirty price (semi-annual coupons)
            semi_annual_coupon = (bond['coupon'] / 100) / 2  # convert to decimal, calculate semi-annual
            accrued_interest = semi_annual_coupon * (accrued_days / total_days)

        # dirty price = clean price + accrued interest (per $100 face value)
        bond['dirty_price'] = bond['close_price'] + accrued_interest

    return bonds

jan6bonds = [
    {
        'coupon': 3.5,
        'maturity_date': '8/1/2025',
        'coupon_payment_start_date': '8/1/2023',
        'close_price': 100.26
    },
    {
        'coupon': 4.5,
        'maturity_date': '2/1/2026',
        'coupon_payment_start_date': '2/1/2024',
        'close_price': 101.58
    },
    {
        'coupon': 4.0,
        'maturity_date': '8/3/2026',
        'coupon_payment_start_date': '8/1/2024',
        'close_price': 101.54
    },
    {
        'coupon': 3.0,
        'maturity_date': '2/1/2027',
        'coupon_payment_start_date': '2/1/2025',
        'close_price': 100.11
    },
    {
        'coupon': 3.25,
        'maturity_date': '8/24/2027',
        'coupon_payment_start_date': '2/24/2023',
        'close_price': 101.02
    },
    {
        'coupon': 3.5,
        'maturity_date': '3/1/2028',
        'coupon_payment_start_date': '3/1/2023',
        'close_price': 101.8
    },
    {
        'coupon': 3.25,
        'maturity_date': '9/1/2028',
        'coupon_payment_start_date': '9/1/2023',
        'close_price': 101.1
    },
    {
        'coupon': 4.0,
        'maturity_date': '3/1/2029',
        'coupon_payment_start_date': '3/1/2024',
        'close_price': 104.07
    },
    {
        'coupon': 3.5,
        'maturity_date': '9/1/2029',
        'coupon_payment_start_date': '9/1/2024',
        'close_price': 102.22
    },
    {
        'coupon': 2.75,
        'maturity_date': '3/1/2030',
        'coupon_payment_start_date': '3/1/2025',
        'close_price': 98.81
    }

]

settlement_date_str = '1/6/2025'

updated_bonds = calculate_dirty_prices(jan6bonds, settlement_date_str)

# for bond in updated_bonds:
#     print(f"Coupon: {bond['coupon']}% | Clean: {bond['close_price']:.2f} | " +
#           f"Dirty: {bond['dirty_price']:.4f}")

In [32]:


from datetime import date
from dateutil.relativedelta import relativedelta

def year_diff_actual(start_date_str, end_date_str):
    start = datetime.strptime(start_date_str, "%m/%d/%Y").date()
    end = datetime.strptime(end_date_str, "%m/%d/%Y").date()

    total_days = (end - start).days
    if total_days == 0:
        return 0.0

    # Iterate year-by-year to handle leap years
    years = 0.0
    current = start
    while current < end:
        next_year = date(current.year + 1, current.month, current.day)
        if next_year > end:
            days_in_year = 366 if current.year % 4 == 0 else 365  # leap year check
            days_remaining = (end - current).days
            years += days_remaining / days_in_year
            break
        else:
            years += 1.0
            current = next_year
    return years

In [33]:
# templatebonds = []
# i=1
# for bond in updated_bonds:
#     maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
#     settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
#     templatebonds.append(tuple((bond['dirty_price'],
#                                bond['coupon'],
#                                0.5*i)))
#     i += 1
# templatebonds

In [34]:
templatebonds = []

for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
templatebonds

cleanedjan6bonds = templatebonds.copy()

## Template Bootstrap Function

In [35]:
# def bootstrap_yield_curve(bonds):
#     spot_rates = np.zeros(len(bonds))

#     for i, (price, coupon_rate, maturity) in enumerate(sorted(bonds, key=lambda x: x[2])):
#         cash_flows = np.array([coupon_rate] * int(maturity - 1) + [100 + coupon_rate])
#         time_periods = np.arange(1, maturity + 1)
#         # Use previously calculated spot rates for discounted cash flows
#         if i == 0:
#             discounted_cash_flows = cash_flows / (1 + spot_rates[i])**time_periods
#         else:
#             discounted_cash_flows = [cf / (1 + spot_rates[j])**time_periods[j] for j, cf in enumerate(cash_flows)]
#             discounted_cash_flows = np.sum(discounted_cash_flows)

#         residual = price - discounted_cash_flows
#         if residual <= 0:
#             # Handle cases where residual is too low
#             print(f"Warning: Residual for bond with maturity {maturity} is too low. Adjusting spot rate calculation.")
#             spot_rate = spot_rates[i-1]  # Use previous spot rate as an approximation
#         else:
#             spot_rate = ((100 / residual)**(1 / maturity)) - 1
#         spot_rates[i] = spot_rate

#     return spot_rates

## Example Bond

In [36]:
# bonds = [
#     (195, 1, 1),
#     (190, 1.5, 2),
#     (188, 2, 3)
# ]

## Bootstrapping spot rates

In [37]:
sorted_bonds = sorted(templatebonds, key=lambda x: x[2])
sorted_bonds

[(100.27515068493152, 3.5, 0.5671232876712329),
 (101.5994794520548, 4.5, 1.0712328767123287),
 (101.55731506849315, 4.0, 1.5726027397260274),
 (100.11, 3.0, 2.0712328767123287),
 (101.03202054794521, 3.25, 2.6301369863013697),
 (101.81217808219178, 3.5, 3.150273224043716),
 (101.11130821917807, 3.25, 3.6530054644808745),
 (104.08391780821917, 4.0, 4.147945205479452),
 (102.23217808219178, 3.5, 4.652054794520548),
 (98.81, 2.75, 5.147945205479452)]

In [38]:
import math

def bootstrap_yield_curve(bonds, face_value=100.0):
    """
    Bootstraps (annualized) spot rates from a list of bonds.

    Returns -
    spot_rates : dict
        Maps each maturity time T (in years) to the annualized spot rate r(T).
    discount_factors : dict
        Maps each maturity time T (in years) to the discount factor D(T).
        This simplifies the coupon calculations
    """


    # Sort bonds by ascending maturity
    bonds_sorted = sorted(bonds, key=lambda x: x[2])

    discount_factors = {}  # time -> discount factor D(T)
    spot_rates = {}        # time -> annualized spot rate r(T)

    freq = 2 # 2 coupon payments per year

    # Maintain a global list of coupon times for which we have solved discount factors.
    # This assumes we get coupon payments from each prior solved time.
    solved_times = []

    for (price, coupon_rate, maturity) in bonds_sorted:


        # each coupon payment in dollars:
        coupon_payment = (coupon_rate * face_value) / freq

        # use every coupon date we have solved that is less than the current bond's maturity,
        # then add the current bond's maturity as the final coupon date.
        coupon_schedule = [t for t in solved_times if t < maturity]
        coupon_schedule.append(maturity)
        coupon_schedule = sorted(coupon_schedule)

        # Now we bootstrap to solve the spot rates iteratively

        known_value = 0.0
        unknown_time = None
        unknown_payment = 0.0

        for i, t in enumerate(coupon_schedule):
            if i < len(coupon_schedule) - 1:
                # For a coupon date that is already solved, its cash flow is coupon_payment.
                # if t not in discount_factors:
                #     raise ValueError(f"Expected discount factor for time {t} but it was not found.")
                # iteratively add coupon payments
                known_value += coupon_payment * discount_factors[t]
            else:
                # The final coupon date (which should be the current bond's maturity) has cash flow:
                # coupon_payment + face_value
                unknown_time = t
                unknown_payment = coupon_payment + face_value

        # discount factor
        D_unknown = (price - known_value) / unknown_payment
        print(D_unknown)

        # If the computed discount factor is not in (0,1),
        # then we just take the previous solved discount factor,
        # similar to what was done in template
        if D_unknown <= 0 or D_unknown >= 1:
            if solved_times:
                previous_D = discount_factors[solved_times[-1]]
                print(f"Warning: Computed D({unknown_time}) = {D_unknown:.6f} is not in (0,1); "+
                      f"using previous discount factor D({solved_times[-1]}) = {previous_D:.6f} instead."+
                      "Similar to original implementation of bootstrapping")
                D_unknown = previous_D

        discount_factors[unknown_time] = D_unknown
        solved_times.append(unknown_time)
        solved_times = sorted(set(solved_times))

        r_spot = freq * ((1.0 / D_unknown) ** (1.0 / (freq * unknown_time)) - 1.0)
        # print(r_spot)
        # print("\n")
        spot_rates[unknown_time] = r_spot

    return spot_rates


In [39]:
# Collate spot rates for graphing
spot_rates_by_day = {}
spot_rates_by_day['1/6/2025'] = bootstrap_yield_curve(templatebonds)

0.36463691158156913
0.06017284414215921
0.05531787974582495
0.11236341871826801
0.018103711399711366
-0.018334203256823427
-0.0040083592047584545
-0.08425506623057302
-0.05136856138354512
0.0206154882605181


In [40]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/7/2025'
jan7bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.25},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.58},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.57},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 100.14},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 101.04},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.82},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 101.14},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 104.01},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 102.14},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 98.6}]

updated_bonds = calculate_dirty_prices(jan7bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
cleanedjan7bonds = templatebonds.copy()
spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan7bonds)


0.3646008966376089
0.06019815691158158
0.05542537982565383
0.11242533997509344
0.01808198636067134
-0.01834830537001401
-0.003926996100225162
-0.08451706363043353
-0.0517048247858411
0.01970227596388195


In [41]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/8/2025'
jan8bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.24},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.56},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.53},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 100.08},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.94},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.7},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.99},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.9},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 102.04},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 98.53}]

updated_bonds = calculate_dirty_prices(jan8bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
cleanedjan8bonds = templatebonds.copy()
spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan8bonds)


0.36456488169364876
0.060161931219465496
0.05534057221317497
0.11227956892422646
0.017888832750202633
-0.018468701189498234
-0.004071483335827787
-0.08429517824641114
-0.05138374553079421
0.02014198443069368


In [42]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/9/2025'
jan9bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.25},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.58},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.55},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 100.1},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 101.02},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.72},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.94},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.87},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 101.98},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 98.54}]

updated_bonds = calculate_dirty_prices(jan9bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
cleanedjan9bonds = templatebonds.copy()

spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan9bonds)


0.36460159402241593
0.06019843280007666
0.05535879490372641
0.11230470696426857
0.018121768605823426
-0.01861804099429006
-0.0046221823490708676
-0.08493840108749699
-0.05226869015086203
0.019442311109702418


In [43]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/10/2025'
jan10bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.19},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.46},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.46},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 99.96},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.72},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.52},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.79},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.53},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 101.59},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 97.96}]

updated_bonds = calculate_dirty_prices(jan10bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
cleanedjan10bonds = templatebonds.copy()


spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan10bonds)


0.3643837608966376
0.059980388926142346
0.055349744867005106
0.11201166318612894
0.017436089742829522
-0.01843900614343454
-0.0038874969055008738
-0.08420803085498983
-0.05147154110701809
0.019412311974174732


In [44]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/13/2025'
jan13bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.15},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.35},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.2},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 99.67},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.46},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.02},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.19},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 102.99},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 103.01},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 97.55}]

updated_bonds = calculate_dirty_prices(jan13bonds, settlement_date_str)



templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))

cleanedjan13bonds = templatebonds.copy()

spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan13bonds)


0.36423935242839356
0.05974304052112267
0.05473867867292526
0.11144735702653515
0.017410568305362816
-0.019249000552512403
-0.005176658468008803
-0.08491780593835391
-0.04525099459002325
0.01866136989822651


In [45]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33
#data problems....
settlement_date_str = '1/14/2025'
jan14bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.15},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.35},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.2},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 99.19}, #data problems...
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.46},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.02},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.19},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 102.99},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 103.01},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 97.29}]

updated_bonds = calculate_dirty_prices(jan14bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))

cleanedjan14bonds = templatebonds.copy()

spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan14bonds)


0.364239701120797
0.059743178465370236
0.054738719545294905
0.10952704052112266
0.018599348316827812
-0.018783464324246366
-0.005459701327881378
-0.08601514133355233
-0.047054947474553924
0.015236884547159782


In [48]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/15/2025'
jan15bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.21},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.46},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.38},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 99.89},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.55},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.36},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.62},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.43},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 103.01},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 97.84}]

updated_bonds = calculate_dirty_prices(jan15bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))

cleanedjan15bonds = templatebonds.copy()

spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan15bonds)


0.3644582316313823
0.05993072899703035
0.05506836414726825
0.11188560513459146
0.01702703164719602
-0.01851528064242385
-0.0037901021208261096
-0.08346633620261017
-0.04502143196888896
0.020312685810064415


In [49]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/16/2025'
jan16bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.25},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.46},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.38},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 100.03},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 100.73},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.58},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 100.92},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.8},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 103.01},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 98.29}]

updated_bonds = calculate_dirty_prices(jan16bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))

cleanedjan16bonds = templatebonds.copy()


spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan16bonds)


0.3646040348692403
0.05983016764057857
0.05503856818980111
0.11243633758022804
0.0173625937654431
-0.018288766805907554
-0.0034128550061231114
-0.08328121405535105
-0.046235536721753505
0.020908271148542122


In [50]:
#rows 7, 11, 16, 19, 23, 25, 27, 28, 31, 33

settlement_date_str = '1/17/2025'
jan17bonds = [
    {'coupon': 3.5, 'maturity_date': '8/1/2025',
     'coupon_payment_start_date': '8/1/2023', 'close_price': 100.26},
    {'coupon': 4.5, 'maturity_date': '2/1/2026',
     'coupon_payment_start_date': '2/1/2024', 'close_price': 101.54},
    {'coupon': 4.0, 'maturity_date': '8/3/2026',
     'coupon_payment_start_date': '8/1/2024', 'close_price': 101.47},
    {'coupon': 3.0, 'maturity_date': '2/1/2027',
     'coupon_payment_start_date': '2/1/2025', 'close_price': 100.07},
    {'coupon': 3.25, 'maturity_date': '8/24/2027',
     'coupon_payment_start_date': '2/24/2023', 'close_price': 101},
    {'coupon': 3.5, 'maturity_date': '3/1/2028',
     'coupon_payment_start_date': '3/1/2023','close_price': 101.65},
    {'coupon': 3.25, 'maturity_date': '9/1/2028',
     'coupon_payment_start_date': '9/1/2023', 'close_price': 101.02},
    {'coupon': 4.0, 'maturity_date': '3/1/2029',
     'coupon_payment_start_date': '3/1/2024', 'close_price': 103.9},
    {'coupon': 3.5, 'maturity_date': '9/1/2029',
     'coupon_payment_start_date': '9/1/2024', 'close_price': 103.01},
    {'coupon': 2.75, 'maturity_date': '3/1/2030',
     'coupon_payment_start_date': '3/1/2025', 'close_price': 98.4}]

updated_bonds = calculate_dirty_prices(jan17bonds, settlement_date_str)

templatebonds = []
for bond in updated_bonds:
    maturity = datetime.strptime(bond['maturity_date'], '%m/%d/%Y').date()
    settlement_date = datetime.strptime(settlement_date_str, '%m/%d/%Y').date()
    templatebonds.append(tuple((bond['dirty_price'],
                               bond['coupon'],
                               year_diff_actual(settlement_date_str, bond['maturity_date']))))
cleanedjan17bonds = templatebonds.copy()

spot_rates_by_day[settlement_date_str] = bootstrap_yield_curve(cleanedjan17bonds)


0.3646407471980075
0.06005128460580518
0.05516704729060891
0.11236455254334703
0.01819679946233372
-0.01876488205747483
-0.0042590944577637485
-0.08482560905760901
-0.04855877194011198
0.01877453060822063


In [25]:
spot_rates_by_day.keys()

dict_keys(['1/6/2025', '1/7/2025', '1/8/2025', '1/9/2025', '1/10/2025', '1/13/2025', '1/14/2025', '1/15/2025', '1/16/2025', '1/17/2025'])

In [26]:
spot_rates = bootstrap_yield_curve(templatebonds)
# print("Spot Rates:", spot_rates)




0.3646407471980075
0.06005128460580518
0.05516704729060891
0.11236455254334703
0.01819679946233372
-0.01876488205747483
-0.0042590944577637485
-0.08482560905760901
-0.04855877194011198
0.01877453060822063


## Plotting the yield curve

In [ ]:
# maturities = [bond[2] for bond in templatebonds]
# fig = go.Figure(go.Scatter(x=maturities, y=list(spot_rates.values()), mode='lines+markers'))
# fig.update_layout(title='Bootstrapped Yield Curve', xaxis_title='Maturity (Years)', yaxis_title='Spot Rate (%)')
# fig.show()

In [28]:
fig = go.Figure()

for day, curve in spot_rates_by_day.items():
    # If the maturities are not guaranteed to be in order, sort them.
    maturities = sorted(curve.keys())
    # Extract the corresponding spot rates (convert to percentages if desired)
    spot_values = [curve[t] for t in maturities]

    fig.add_trace(go.Scatter(
        x=maturities,
        y=spot_values,
        mode='lines+markers',
        name=str(day)
    ))

fig.update_layout(
    title='Bootstrapped Spot Curve for Multiple Days',
    xaxis_title='Maturity (Years)',
    yaxis_title='Spot Rate (%)'
)
fig.show()


In [51]:
#Calculating yield to maturity curves

import math
from scipy.optimize import newton

def bond_price_from_ytm(y, coupon_rate, maturity):
    """
    Compute the bond price given an annualized yield 'y' (compounded freq times per year),
    an annual coupon rate (as a decimal), a maturity in years, a face value, and coupon frequency.
    """
    face_value = 100.0
    freq = 2

    coupon = (coupon_rate * face_value) / freq

    # N is number of full coupon periods before maturity
    N = int(math.floor(maturity * freq))


    # Sum coupon payments for full periods (each coupon paid in 6 months is considered 1 step)
    price = 0.0
    for i in range(1, N + 1):
        price += coupon / ((1 + y/freq)**(i))

    # discount the final payment (coupon + face value) at the exact time (which may be fractional)
    final_time = maturity * freq
    price_final = (coupon + face_value) / ((1 + y/freq)**final_time)

    # Prevent double counting. If the maturity*freq is exactly an integer, then we already counted a coupon for period N.
    # In that case, subtract the coupon for period N and replace it with the final payment.
    if abs(final_time - N) < 1e-8:
        price = price - coupon / ((1 + y/freq)**(N)) + price_final
    else:
        price += price_final

    return price



def ytm_for_bond(dirty_price, coupon_rate, maturity, face_value=100, freq=2, guess=0.05):
    """
    Given a bond's dirty price, annual coupon rate, and maturity (in years),
    solve for its yield to maturity (YTM) on an annualized basis (compounded freq times per year)
    using Newton's method (secant method variant).
    """
    face_value=100
    freq=2
    # Define the function whose root we wish to find: f(y) = computed_price(y) - dirty_price
    f = lambda y: bond_price_from_ytm(y, coupon_rate, maturity) - dirty_price

    try:
        ytm = newton(f, guess)  # Using a default guess (5%)
    except Exception as e:
        print(f"Could not solve YTM for bond (Price={dirty_price}, Coupon={coupon_rate}, Maturity={maturity}): {e}")
        ytm = None
    return ytm

In [55]:
bond_data_by_day = {
    '1/6/2025':cleanedjan6bonds,
    '1/7/2025':cleanedjan7bonds,
    '1/8/2025':cleanedjan8bonds,
    '1/9/2025':cleanedjan9bonds,
    '1/10/2025':cleanedjan10bonds,
    '1/13/2025':cleanedjan13bonds,
    '1/14/2025':cleanedjan14bonds,
    '1/15/2025':cleanedjan15bonds,
    '1/16/2025':cleanedjan16bonds,
    '1/17/2025':cleanedjan17bonds}

# Compute the yield curves for each day:
yield_curves_by_day = {}

for day, bonds in bond_data_by_day.items():
    maturities = []
    ytms = []
    for bond in bonds:
        dirty_price, coupon_rate, maturity = bond
        ytm = ytm_for_bond(dirty_price, coupon_rate, maturity, face_value=100, freq=2)
        if ytm is not None:
            maturities.append(maturity)
            ytms.append(ytm)
    # Sort by maturity in case the bonds are not in order
    sorted_pairs = sorted(zip(maturities, ytms), key=lambda x: x[0])
    if sorted_pairs:
        maturities_sorted, ytms_sorted = zip(*sorted_pairs)
        yield_curves_by_day[day] = (maturities_sorted, ytms_sorted)

fig = go.Figure()

for day, (maturities, ytms) in yield_curves_by_day.items():
    fig.add_trace(go.Scatter(
        x=list(maturities),
        y=list(ytms),
        mode='lines+markers',
        name=day
    ))

fig.update_layout(
    title='Yield to Maturity Curves for Jan 6 - Jan 17',
    xaxis_title='Maturity (Years)',
    yaxis_title='Yield to Maturity (%)'
)
fig.show()

In [64]:
import numpy as np
from scipy.interpolate import CubicSpline
import plotly.graph_objects as go

# For storing the forward curves for each day (Jan 6 to Jan 17).
forward_curves_by_day = {}

# For plotting the interpolated spot curves (if desired)
spot_spline_curves_by_day = {}

# Define target maturities for interpolation.
target_maturities = [1, 2, 3, 4, 5]  # years

for day, spot_dict in spot_rates_by_day.items():
    # Sort the bootstrapped maturities and corresponding spot rates.
    boot_maturities = np.array(sorted(spot_dict.keys()))
    boot_rates = np.array([spot_dict[t] for t in boot_maturities])

    # Interpolate the spot rates at the target maturities.
    interp_spot_rates = np.interp(target_maturities, boot_maturities, boot_rates)

    # r(1) is the spot rate at T = 1 year.
    r1 = interp_spot_rates[0]  # interpolated at time T=1

    # compute forward rates f(1,T) for T = 2,3,4,5.
    forward_periods = []
    forward_rates = []
    for i, T in enumerate(target_maturities):
        if T > 1:
            rT = interp_spot_rates[i]  # spot rate at maturity T
            # Compute f(1, T) using the relation:
            # (1 + r(T))^T = (1 + r(1)) * (1 + f(1,T))^(T-1)
            fwd_rate = (( (1 + rT)**T / (1 + r1) )**(1/(T - 1))) - 1
            forward_periods.append(T - 1)  # forward period length is T-1 years
            forward_rates.append(fwd_rate)

    forward_curves_by_day[day] = (forward_periods, forward_rates)

# Now plot all forward curves on the same figure using Plotly.
fig = go.Figure()

for day, (fwd_periods, fwd_rates) in forward_curves_by_day.items():
    fig.add_trace(go.Scatter(
        x=fwd_periods,
        y=fwd_rates,
        mode='lines+markers',
        name=day
    ))

fig.update_layout(
    title='1-Year Forward Curves (Using Linear Interpolation of the Spot Curve)',
    xaxis_title='Forward Period Length (Years) [e.g., 1 = 1yr-1yr, 2 = 1yr-2yr, 4 = 1yr-4yr]',
    yaxis_title='Forward Rate (%)',
    legend_title='Date'
)

fig.show()

In [66]:
bond_data_by_day

{'1/6/2025': [(100.27515068493152, 3.5, 0.5671232876712329),
  (101.5994794520548, 4.5, 1.0712328767123287),
  (101.55731506849315, 4.0, 1.5726027397260274),
  (100.11, 3.0, 2.0712328767123287),
  (101.03202054794521, 3.25, 2.6301369863013697),
  (101.81217808219178, 3.5, 3.150273224043716),
  (101.11130821917807, 3.25, 3.6530054644808745),
  (104.08391780821917, 4.0, 4.147945205479452),
  (102.23217808219178, 3.5, 4.652054794520548),
  (98.81, 2.75, 5.147945205479452)],
 '1/7/2025': [(100.26524657534246, 3.5, 0.5643835616438356),
  (101.59960273972602, 4.5, 1.0684931506849316),
  (101.58742465753424, 4.0, 1.56986301369863),
  (100.14, 3.0, 2.0684931506849313),
  (101.05210958904111, 3.25, 2.6273972602739724),
  (101.83227397260274, 3.5, 3.1475409836065573),
  (101.15139726027397, 3.25, 3.650273224043716),
  (104.02402739726028, 4.0, 4.145205479452055),
  (102.15227397260274, 3.5, 4.64931506849315),
  (98.6, 2.75, 5.145205479452055)],
 '1/8/2025': [(100.25534246575342, 3.5, 0.561643835

In [71]:
# yield_curves_by_day
# forward_curves_by_day

In [72]:
yield_curves_by_day.keys()

dict_keys(['1/6/2025', '1/7/2025', '1/8/2025', '1/9/2025', '1/10/2025', '1/13/2025', '1/14/2025', '1/15/2025', '1/16/2025', '1/17/2025'])

In [84]:

# ------------------------------
# Yields

sorted_days = sorted(yield_curves_by_day.keys())

# Create a 2D array: rows = days, columns = maturities (i.e., each column is a time series for one yield).
yield_list = []
for day in sorted_days:
    maturities, yields_ = yield_curves_by_day[day]
    yield_list.append(yields_)
yield_array = np.array(yield_list)[:,:5] # shape: (num_days, 5).
# Take only first 5 maturities

# Compute daily log-returns for each yield (for j = 0,...,num_days-2)
# The resulting array will have shape (num_days - 1, 5)
yield_log_returns = np.log(yield_array[1:,:] / yield_array[:-1,:])

# compute the covariance matrix across the 5 yield series.
# np.cov expects rows = variables, pass the transpose.
cov_yields = np.cov(yield_log_returns.T)

print("Covariance matrix for yield log-returns:")
print(cov_yields)

# ------------------------------
# Forward Rates

# Do a similar process for the forward rates.
sorted_days_forward = sorted(forward_curves_by_day.keys())

forward_list = []
for day in sorted_days_forward:
    periods, fwd_rates = forward_curves_by_day[day]
    forward_list.append(fwd_rates)
forward_array = np.array(forward_list)  # shape: (num_days, 4)

# Compute log-returns for forward rates.
forward_log_returns = np.log(forward_array[1:,:] / forward_array[:-1,:])

# Covariance matrix (4 x 4) for the forward rate log-returns.
cov_forwards = np.cov(forward_log_returns.T)

print("\nCovariance matrix for forward rate log-returns:")
print(cov_forwards)

Covariance matrix for yield log-returns:
[[5.29628845e-04 1.44262402e-04 5.87804947e-05 3.24314329e-05
  1.14180354e-05]
 [1.44262402e-04 3.95568163e-05 1.65279595e-05 9.88922335e-06
  3.70882368e-06]
 [5.87804947e-05 1.65279595e-05 7.63709016e-06 5.84560287e-06
  2.49682253e-06]
 [3.24314329e-05 9.88922335e-06 5.84560287e-06 1.17779871e-05
  2.77175139e-06]
 [1.14180354e-05 3.70882368e-06 2.49682253e-06 2.77175139e-06
  2.47438121e-06]]

Covariance matrix for forward rate log-returns:
[[ 1.79590231e+00  3.80012077e-02  3.16237859e-02  1.38309281e-02]
 [ 3.80012077e-02  1.76429765e-03  1.52097740e-03 -1.09837515e-03]
 [ 3.16237859e-02  1.52097740e-03  1.31285650e-03 -9.94788917e-04]
 [ 1.38309281e-02 -1.09837515e-03 -9.94788917e-04  2.32264560e-03]]


In [86]:
# Compute eigenvalues and eigenvectors for yields.
eigvals_y, eigvecs_y = np.linalg.eig(cov_yields)
print("Eigenvalues for yield covariance matrix:\n", eigvals_y)
print("Eigenvectors for yield covariance matrix:\n", eigvecs_y)

# Compute eigenvalues and eigenvectors for forward rates.
eigvals_f, eigvecs_f = np.linalg.eig(cov_forwards)
print("Eigenvalues for forward rate covariance matrix:\n", eigvals_f)
print("Eigenvectors for forward rate covariance matrix:\n", eigvecs_f)

Eigenvalues for yield covariance matrix:
 [5.77853130e-04 1.09472168e-05 2.00451229e-06 1.69140992e-08
 2.53346642e-07]
Eigenvectors for yield covariance matrix:
 [[ 0.95721757 -0.11390744 -0.08080797 -0.19965432  0.15610205]
 [ 0.26107773  0.08545956  0.16054652  0.89616773 -0.3092625 ]
 [ 0.10695706  0.23418484  0.35975934 -0.39567924 -0.80481601]
 [ 0.06061109  0.92696395 -0.34947276 -0.00127833  0.12219364]
 [ 0.02143438  0.25615827  0.84624872 -0.02139762  0.46618533]]
Eigenvalues for forward rate covariance matrix:
 [1.79737043e+00 3.84457742e-03 8.69688114e-05 1.31763708e-07]
Eigenvectors for forward rate covariance matrix:
 [[-9.99591329e-01 -1.23348639e-02 -2.57865119e-02  2.84641318e-04]
 [-2.11650165e-02  4.91843502e-01  5.77987938e-01 -6.50824058e-01]
 [-1.76138087e-02  4.37223600e-01  4.82018618e-01  7.59067407e-01]
 [-7.67918649e-03 -7.52843506e-01  6.57968883e-01  1.56408202e-02]]
